In [5]:
""""We are extracting each frame of the video, with the help of segmentation. 
we separate the background and foreground of the image. 
We replace the foreground of a particular color with the background 
which gives the illusion of getting disappeared"""

import cv2

import numpy as np
import time

fourcc=cv2.VideoWriter_fourcc(*'XVID')
#FourCC is a 4-byte code used to specify the video codec. The list of available codes can be found in fourcc.org
out=cv2.VideoWriter('newoutput.avi', fourcc, 20.0, (640,  480))

cap = cv2.VideoCapture(0)

time.sleep(3)
count=0
background=0

#store the background image for each frame.
#For capturing the current frame, we have used Cap.read() function and stored it in the variable named ‘frame’. 
#The variable ‘var’ is used to store Boolean True/False, var will give True value if the frame is read correctly otherwise it will give False
#for loop so that we have several frames for the background as averaging over multiple frames also reduces noise.

for i in range(0,60):
  var,background=cap.read()
frame = cv2.flip(background, 0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        #print("Can't receive frame (stream end?). Exiting ...")
        break
    frame = cv2.flip(frame, 1)

    """convert the RGB (red-blue-green) to HSV(hue-saturation-value) because RGB values are highly sensitive to illumination. After the conversion of RGB to HSV, it is time to specify the range of color to detect red color in the video.
In Hue value, the red color is in range 0-30 as well as 150-180. To avoid the detection of skin as the red color we will use 0-10 and 170-180 range and combine both with the OR operator"""

    hsv=cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)

    #generate mark to detect red color

    lower_red=np.array([0,120,50])
    upper_red=np.array([10,255,255])
    mask1=cv2.inRange(hsv,lower_red,upper_red)

    lower_red=np.array([170,120,70])
    upper_red=np.array([180,255,255])
    mask2=cv2.inRange(hsv,lower_red,upper_red)

    mask1=mask1+mask2
    #MorphologyEx- perform advanced morphological transformations using an erosion and dilation as basic operations.
    #open and dilate the mask image
    mask1=cv2.morphologyEx(mask1,cv2.MORPH_OPEN,np.ones((3,3),np.uint8))
    mask2=cv2.morphologyEx(mask1,cv2.MORPH_DILATE,np.ones((3,3),np.uint8))
    
    #create inverted mask to segment out red color from the frame
    mask2=cv2.bitwise_not(mask1)
    
    #segment the red color part out of the frame using bitwise and with the inverted mask
    res1=cv2.bitwise_and(frame,frame,mask=mask2)

    #create an image using the static static background pixels only for masked pixels
    res2=cv2.bitwise_and(background,background,mask=mask1)

    finalOp=cv2.addWeighted(res1,1,res2,1,0)
    out.write(finalOp)
    cv2.imshow("magic",finalOp)
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break
cap.release()
out.release()
cv2.destroyAllWindows()

In [ ]:
cap.release()
out.release()
cv2.destroyAllWindows()